In [1]:
import pandas as pd

dfs_k = []
dfs_theta = []
for i in range(1, 6):
    dfs_k.append(pd.read_csv(f'./raw_data/kz_{i}.csv'))
    dfs_theta.append(pd.read_csv(f'./raw_data/Theta_{i}.csv'))


使用Python读取提供的CSV文件。


In [ ]:
import numpy as np

# 假设true_values为已知的参考高程值矩阵
# 这里我们用随机数生成一个示例，实际应用中应使用实际参考数据
true_values = np.random.uniform(150, 300, size=(805, 2001))


In [2]:

def estimate_height(phi1, phi2, k1, k2):
    n = np.round((phi1 - phi2) / (2 * np.pi))
    h = (phi1 - phi2 + 2 * np.pi * n) / (k1 - k2)
    return h

def calculate_elevation(kz, theta, n):
    return - (1 / kz) * (theta + 2 * n * np.pi)
    

heights = estimate_height(df_theta1['theta'], df_theta2['theta'], df_k1['k'], df_k2['k'])


KeyError: 'theta'

根据问题描述，地形高程 

$$h = \frac{\phi - \theta _k }{k} + 2n\pi $$
其中， phi是测得的相位， theta k是已知参数，k 是基线参数， n 是相位缠绕整数。

对于两组数据，可以使用联合方程组来估计高程

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(heights.reshape(805, 2001), cmap='viridis')
plt.colorbar(label='Height (m)')
plt.show()

# 输出四个顶点的高程值
print("Top-left:", heights[0, 0])
print("Top-right:", heights[0, -1])
print("Bottom-left:", heights[-1, 0])
print("Bottom-right:", heights[-1, -1])


> 使用matplotlib等工具绘制高程色度图。
> 提取四个顶点的高程值并输出。

In [ ]:
from sklearn.metrics import mean_squared_error

def calculate_mse(heights, true_values):
    return mean_squared_error(true_values, heights)

# 逐步增加数据组数量，评估误差和计算复杂度
errors = []
for num_groups in range(2, 6):
    combined_heights = []
    for i in range(num_groups):
        for j in range(i + 1, num_groups):
            heights = estimate_height(dfs_theta[i]['theta'], dfs_theta[j]['theta'], dfs_k[i]['k'], dfs_k[j]['k'])
            combined_heights.append(heights)
    combined_heights = np.mean(combined_heights, axis=0)
    error = mean_squared_error(true_values, combined_heights)
    errors.append(error)

optimal_groups = np.argmin(errors) + 2
print("Optimal number of groups:", optimal_groups)


误差分析与优化:

使用更多数据组可以提高高程估计的精度，但会增加计算复杂度。
通过交叉验证或最小二乘法等方法评估多组数据对精度的影响。
优化选择数据组的数量，使得在保证精度的情况下计算复杂度最低。

In [ ]:
import statsmodels.api as sm

# 使用简化的OLS模型进行误差分析
# 这里我们假设combined_heights已经通过最优组合计算得出
combined_heights = np.mean([estimate_height(dfs_theta[0]['theta'], dfs_theta[1]['theta'], dfs_k[0]['k'], dfs_k[1]['k']),
                            estimate_height(dfs_theta[1]['theta'], dfs_theta[2]['theta'], dfs_k[1]['k'], dfs_k[2]['k'])], axis=0)

# 展平数组用于线性回归模型
X = combined_heights.flatten()
y = true_values.flatten()

# 添加常数项用于回归分析
X = sm.add_constant(X)

# 进行回归分析
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

# 打印回归结果摘要
print(results.summary())


误差分析:

分析误差来源（测量误差、模型误差、相位缠绕误差等）。
使用统计方法分析误差对高程估计的影响。